# Frame Unwrapping

## Context

At time-of-flight neutron sources recording event-mode, time-stamps of detected neutrons are written to files in an `NXevent_data` group.
This contains two main time components, `event_time_zero` and `event_time_offset`.
The sum of the two would typically yield the absolute detection time of the neutron.
For computation of wavelengths or energies during data-reduction, a time-of-flight is required.
In principle the time-of-flight could be equivalent to `event_time_offset`, and the emission time of the neutron to `event_time_zero`.
Since an actual computation of time-of-flight would require knowledge about chopper settings, detector positions, and whether the scattering of the sample is elastic or inelastic, this may however not be the case in practice.
Instead, the data acquisition system may, e.g., record the time at which the proton pulse hits the target as `event_time_zero`, with `event_time_offset` representing the offset since then.

We refer to the process of "unwrapping" these time stamps into an actual time-of-flight as *frame unwrapping*, since `event_time_offset` "wraps around" with the period of the proton pulse and neutrons created by different proton pulses may be recorded with the *same* `event_time_zero`.
The figures in the remainder of this document will clarify this.

## Default mode

Often there is a 1:1 correspondence between source pulses and neutron pulses propagated to the sample and detectors.

In [ ]:
from frameunwrapping import default_frame_diagram

default_frame_diagram().show()

In the figure above the index `i` labels source pulses.
We define:

- $T_0^i$ is the `event_time_zero` recorded in an `NXevent_data` group.
  These times are indicated by the vertical dotted lines.
- $T_0^{i+1} = T_0^i + L_0$ where the frame length $L_0$ is defined by $L_0 = 1/f_0$, given a source frequency $f_0$.
- $\Delta T_0$ is the offset from $T_0^i$ at which the neutrons are "emitted".
  This may be zero (or half the pulse length) if the full pulse length is used, but choppers such as resolution choppers may extract a section of the pulse that is not aligned with the start of the full pulse.
  This offset can also be used to take into account a potential difference between the timing system's definition of the pulse time and the actual beginning of the neutron pulse exiting, e.g., the moderator.
- The black solid line within the first pulse (blue) indicates a neutron detected at $T_0^{i+1} + \Delta t$.
  $\Delta t$ is the `event_time_offset` in an `NXevent_data` group.
  This value is recorded for every neutron and gives the offset from the latest (previous) `event_time_zero` ($T_0^j$), i.e., the time difference to the previous vertical dotted line.

To compute the time-of-flight for a neutron, we need to identify which source pulse it originated from.
Consider the shaded vertical band above, indicating the time during which arriving neutrons are associated with $T_0^{i+1}$.
For, e.g., detector 1 when then observe:

- First (small `event_time_offset` $\Delta t$, to the left of the dashed black line) we see the slowest neutrons from N (in this case N=2) source pulses earlier.
- Then (larger `event_time_offset` $\Delta t$, to the right of the dashed black line) we see the fastest neutrons from N-1 (in this case N-1=1) source pulses earlier.
- Typically there is is an intermediate region where no neutrons should be able to traverse the chopper cascade.
  Neutrons detected in this time interval must thus be background from other sources.

To compute the time-of-flight we first add an integer multiple of the frame length to `event_time_offset` (and subtract the equivalent from `event_time_zero`).
Within a given frame (indicated above by a band between two dotted vertical lines, such as the grey shaded band) there is a *pivot time*:
Neutrons with `event_time_offset` *before* the pivot time originated one source frame *before* neutrons *after* the pivot time.
As illustrated in the figure, the pivot time $t_\text{pivot}$ depends on the detector or rather the distance of the detector (or monitor) from the scattering position.

The pivot time can be computed from the properties of the source pulse and the chopper cascade, using the [scippneutron.tof.unwrap](https://scipp.github.io/scippneutron/generated/modules/scippneutron.tof.unwrap.html) module:


In [ ]:
import scipp as sc
import sciline as sl
from scippneutron.tof import unwrap

The module performs the following transformations:

In [ ]:
pl = sl.Pipeline(unwrap.providers())
pl.visualize(unwrap.TofData)

## Pulse-skipping mode

Choppers may be used to skip pulses, for the purpose of a simultaneous study of a wider wavelength range.
Conceptually this looks as follows:

In [ ]:
from frameunwrapping import frame_skipping_diagram

frame_skipping_diagram().show()

The task graph that was given above for the non-pulse-skipping mode is then extended as shown below:

In [ ]:
pl = sl.Pipeline(unwrap.providers(pulse_skipping=True))
pl.visualize(unwrap.TofData)

For illustration, we create some fake event data:

In [ ]:
import numpy as np
from scippneutron.conversion import graph

import scipp as sc

N = 1_000_000
Lmax = 160 * sc.Unit('m')
tmax = 700 * sc.Unit('ms')
pulse_period = 71.0 * sc.Unit('ms')
frame_period = 2 * pulse_period
wavmin = 5.0 * sc.Unit('angstrom')
wavmax = 8.0 * sc.Unit('angstrom')
# This fake data is for a frame_offset of 0.

rng = np.random.default_rng(12345)


def fake_frame(frame_number):
    dims = ['event']
    L = sc.array(dims=dims, values=rng.random(N)) * Lmax
    t = sc.array(dims=dims, values=rng.random(N)) * tmax
    table = sc.DataArray(sc.ones(dims=dims, shape=[N]), coords={'Ltotal': L, 'tof': t})
    table = table.transform_coords('wavelength', graph=graph.tof.elastic("tof"))
    da = table.bin(wavelength=1).bins['wavelength', wavmin:wavmax]
    return da.transform_coords(time=lambda tof: tof + frame_number * frame_period)


da = sc.concat([fake_frame(i) for i in range(-2, 8)], 'dummy').bins.concat('dummy')

da = da.bins['time', 0.0 * sc.Unit('ms') : 8 * frame_period]
del da.coords['time']  # avoid problems below with non-monotonic coord transform

Our fake data illustrates what we might observe when placing many neutron monitors along our beamline.
Note that intensities are not modelled correctly, since this is irrelevant here:

In [ ]:
da.hist(Ltotal=100, time=500).plot()

We refer to each of the emitted bundles of neutrons as *frame*.

In practice, NeXus files record event data using `NXevent_data`, and in particular the `event_time_zero` and `event_time_offset` fields.
`event_time_zero` gives the time of the source pulse.
When pulse-skipping is used, the neutrons from each *frame* may nevertheless be assigned to *pulses* in the NeXus files.
That is, while we have a frame-period of $2 \cdot 71~\text{ms}$ (in this example), `event_time_zero` is in steps of $71~\text{ms}$.
The `event_time_offset` that we would obtain from a NeXus file thus records the modulus of the `time`:

In [ ]:
wrapped = da.transform_coords(
    event_time_offset=lambda time: time % pulse_period,
    event_time_zero=lambda time: pulse_period * (time // pulse_period),
)
wrapped.hist(Ltotal=100, event_time_offset=100).plot()

If we consider only neutrons that arrive at the sample or detectors, and split them by their `event_time_zero`, we obtain:

In [ ]:
wrapped.group('event_time_zero').bins['Ltotal', 159.0 * sc.Unit('m') :].hist(
    event_time_offset=100
).plot()

This shows a gap in every other *pulse*, corresponding to the gap between *frames*.
Every other pulse shows no gap, since those pulses are completely within a frame.
Note that all this depends on the detection point and, e.g., different sample-detector distances will lead to gaps at different locations or even pulses.

We can now use [unwrap_frames](../generated/modules/scippneutron.tof.frames.unwrap_frames.rst) to "unwrap" our fake raw data (or actual data from a NeXus file):

In [ ]:
from scippneutron.tof import unwrap_frames

raw = wrapped.copy()
del raw.bins.coords['tof']  # Pretend we do not know 'tof' yet

unwrapped = unwrap_frames(
    raw,
    pulse_period=pulse_period,
    pulse_stride=2,
    lambda_min=wavmin,
    frame_offset=0.0 * sc.Unit('ms'),
    first_pulse_time=0 * pulse_period,
)
unwrapped = unwrapped.hist(Ltotal=100, tof=200)
unwrapped.plot()

At the sample or a detector we would only see this for a particular `Ltotal`:

In [ ]:
unwrapped['Ltotal', -1].plot()

### Failure cases 

At this point it is worth studying some failure cases that we may observe when incorrect parameters are given to `unwrap_frames`.
Note that in all cases below we show a result for the entire `Ltotal` range from 0 to the maximum.
In practice, values will only be available at a few distances (monitors and detectors), so it will be much harder to distinguish the failure cases.

#### Pulse stride 1 instead of 2

In [ ]:
unwrap_frames(
    raw,
    pulse_period=pulse_period,
    pulse_stride=1,
    lambda_min=wavmin,
    frame_offset=0.0 * sc.Unit('ms'),
    first_pulse_time=0 * pulse_period,
).hist(Ltotal=100, tof=200).plot()

#### Pulse stride 3 instead of 2

In [ ]:
unwrap_frames(
    raw,
    pulse_period=pulse_period,
    pulse_stride=3,
    lambda_min=wavmin,
    frame_offset=0.0 * sc.Unit('ms'),
    first_pulse_time=0 * pulse_period,
).hist(Ltotal=100, tof=200).plot()

#### First pulse off by 1

In [ ]:
unwrap_frames(
    raw,
    pulse_period=pulse_period,
    pulse_stride=2,
    lambda_min=wavmin,
    frame_offset=0.0 * sc.Unit('ms'),
    first_pulse_time=1 * pulse_period,
).hist(Ltotal=100, tof=200).plot()

#### Wrong frame offset

In [ ]:
unwrap_frames(
    raw,
    pulse_period=pulse_period,
    pulse_stride=2,
    lambda_min=wavmin,
    frame_offset=10.0 * sc.Unit('ms'),
    first_pulse_time=0 * pulse_period,
).hist(Ltotal=100, tof=200).plot()

#### Bad minimum wavelength

In [ ]:
unwrap_frames(
    raw,
    pulse_period=pulse_period,
    pulse_stride=2,
    lambda_min=0.8 * wavmin,
    frame_offset=0.0 * sc.Unit('ms'),
    first_pulse_time=0 * pulse_period,
).hist(Ltotal=100, tof=200).plot()

## Wavelength-frame multiplication

Frame-unwrapping is related to WFM stitching, so it is performed at the same time.
Without pulse-skipping we have the following task graph:

In [ ]:
pl = sl.Pipeline(unwrap.providers(wfm=True))
pl.visualize(unwrap.TofData)

With pulse-skipping we have the following task graph:

In [ ]:
pl = sl.Pipeline(unwrap.providers(wfm=True, pulse_skipping=True))
pl.visualize(unwrap.TofData)